# Professional AI Agent with Tool Use

Build a production-ready AI agent that represents you professionally with tool integration.
This agent uses function calling to record user details, track unknown questions,
and send real-time notifications via Pushover.

## What You'll Learn
- Function calling (tool use) with OpenAI
- Real-world integrations (Pushover notifications)
- Building deployable Gradio apps
- Professional agent workflows
- Dynamic tool handling without hardcoded if/else statements

## Setup: Pushover Notifications

**Pushover** sends push notifications to your phone in real-time.

### Setup Instructions:
1. Visit https://pushover.net/
2. Click "Login or Signup" (top right) to create a free account
3. On home screen, click "Create an Application/API Token"
4. Name it (e.g., "AI Agent") and click "Create Application"
5. Add to your `.env` file:
   - `PUSHOVER_USER` = key from top-right of home screen (starts with 'u')
   - `PUSHOVER_TOKEN` = key from your application page (starts with 'a')
6. Click "Add Phone, Tablet or Desktop" to install on your device

### Why Pushover?
- Get instant notifications when users interact with your agent
- Know when someone wants to connect with you
- Track questions you can't answer to improve your agent

## Imports and Setup

In [ ]:
"""
Required Files and API Keys:

1. Create .env file with:
   OPENAI_API_KEY=your_openai_key
   PUSHOVER_USER=your_pushover_user_key
   PUSHOVER_TOKEN=your_pushover_token

2. Create 'me' folder with:
   - linkedin.pdf (your LinkedIn profile)
   - summary.txt (your personal summary)

⚠️ All sensitive data is protected by .gitignore
"""

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [ ]:
# Load environment variables
load_dotenv(override=True)
openai = OpenAI()

In [ ]:
# Verify Pushover credentials
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

print("Pushover Configuration:")
print("=" * 50)

if pushover_user:
    print(f"✓ Pushover user found (starts with: {pushover_user[0]})")
else:
    print("✗ Pushover user not found")
    print("  Set PUSHOVER_USER in .env file")

if pushover_token:
    print(f"✓ Pushover token found (starts with: {pushover_token[0]})")
else:
    print("✗ Pushover token not found")
    print("  Set PUSHOVER_TOKEN in .env file")

print("=" * 50)

## Step 1: Pushover Integration

In [ ]:
def push(message):
    """
    Send a push notification via Pushover.
    
    Args:
        message: The notification message to send
    
    Returns:
        Response from Pushover API
    """
    print(f"📱 Push notification: {message}")
    
    if not pushover_user or not pushover_token:
        print("⚠️ Pushover not configured - notification not sent")
        return None
    
    payload = {
        "user": pushover_user,
        "token": pushover_token,
        "message": message
    }
    
    try:
        response = requests.post(pushover_url, data=payload)
        response.raise_for_status()
        print("✓ Notification sent successfully")
        return response
    except Exception as e:
        print(f"✗ Error sending notification: {e}")
        return None

In [ ]:
# Test Pushover (uncomment to send test notification)
# push("Test notification from your AI agent!")

## Step 2: Define Tool Functions

These functions will be called by the AI agent when needed.

In [ ]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    """
    Records user contact information when they express interest.
    
    Args:
        email: User's email address (required)
        name: User's name (optional)
        notes: Additional context from conversation (optional)
    
    Returns:
        Confirmation dict
    """
    notification = f"📧 New contact from {name}\n"
    notification += f"Email: {email}\n"
    notification += f"Notes: {notes}"
    
    push(notification)
    
    return {"recorded": "ok", "message": "Contact details saved"}

In [ ]:
def record_unknown_question(question):
    """
    Records questions the agent couldn't answer.
    
    Args:
        question: The question that couldn't be answered
    
    Returns:
        Confirmation dict
    """
    notification = f"❓ Question I couldn't answer:\n{question}"
    push(notification)
    
    return {"recorded": "ok", "message": "Question logged for review"}

In [ ]:
# Test the functions (uncomment to test)
# record_user_details("test@example.com", "Test User", "Just testing the system")
# record_unknown_question("What's your favorite color?")

## Step 3: Define Tool Schemas for OpenAI

Tool schemas tell the AI what functions are available and how to use them.

In [ ]:
# Schema for recording user details
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            },
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [ ]:
# Schema for recording unknown questions
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            }
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [ ]:
# Combine tools into a list
tools = [
    {"type": "function", "function": record_user_details_json},
    {"type": "function", "function": record_unknown_question_json}
]

print("✓ Tool schemas configured")
print(f"  Available tools: {len(tools)}")

## Step 4: Tool Execution Handler

This function executes tool calls from the AI agent.

In [ ]:
def handle_tool_calls(tool_calls):
    """
    Handles tool calls from the AI model.
    
    This elegant implementation uses globals() to avoid hardcoded if/else statements.
    When adding new tools, just define the function - no need to modify this handler!
    
    Args:
        tool_calls: List of tool calls from OpenAI response
    
    Returns:
        List of tool results in OpenAI format
    """
    results = []
    
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        print(f"🔧 Tool called: {tool_name}", flush=True)
        print(f"   Arguments: {arguments}")
        
        # Get the function from global scope
        tool = globals().get(tool_name)
        
        # Execute the tool
        if tool:
            try:
                result = tool(**arguments)
                print(f"✓ Tool executed successfully")
            except Exception as e:
                result = {"error": str(e)}
                print(f"✗ Tool execution error: {e}")
        else:
            result = {"error": f"Tool {tool_name} not found"}
            print(f"✗ Tool not found: {tool_name}")
        
        # Format result for OpenAI
        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })
    
    return results

## Step 5: Load Personal Data

In [ ]:
# Read LinkedIn PDF
try:
    reader = PdfReader("me/linkedin.pdf")
    linkedin = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            linkedin += text
    print(f"✓ LinkedIn profile loaded ({len(linkedin)} characters)")
except FileNotFoundError:
    print("⚠️ linkedin.pdf not found in 'me' folder")
    linkedin = "No LinkedIn data available."

In [ ]:
# Read personal summary
try:
    with open("me/summary.txt", "r", encoding="utf-8") as f:
        summary = f.read()
    print(f"✓ Summary loaded ({len(summary)} characters)")
except FileNotFoundError:
    print("⚠️ summary.txt not found in 'me' folder")
    summary = "No summary available."

In [ ]:
# Configure your name
name = "Your Name"  # ← CHANGE THIS TO YOUR NAME!

print(f"\n✓ Agent configured for: {name}")

## Step 6: Create System Prompt

In [ ]:
# Build comprehensive system prompt
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

print("\n✓ System prompt created")
print(f"  Total length: {len(system_prompt)} characters")

## Step 7: Main Chat Function with Tool Support

In [ ]:
def chat(message, history):
    """
    Main chat function with tool calling support.
    
    Process:
    1. Send user message to AI
    2. If AI wants to call tools, execute them
    3. Send tool results back to AI
    4. Repeat until AI generates final response
    5. Return response to user
    
    Args:
        message: User's message
        history: Conversation history
    
    Returns:
        Agent's final response
    """
    # Build messages with system prompt and history
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    
    done = False
    iteration = 0
    max_iterations = 10  # Prevent infinite loops
    
    while not done and iteration < max_iterations:
        iteration += 1
        print(f"\n--- Iteration {iteration} ---")
        
        # Call OpenAI with tools
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools
        )
        
        finish_reason = response.choices[0].finish_reason
        print(f"Finish reason: {finish_reason}")
        
        # Check if AI wants to call tools
        if finish_reason == "tool_calls":
            message_with_tools = response.choices[0].message
            tool_calls = message_with_tools.tool_calls
            
            print(f"Tools requested: {len(tool_calls)}")
            
            # Execute tool calls
            results = handle_tool_calls(tool_calls)
            
            # Add tool calls and results to conversation
            messages.append(message_with_tools)
            messages.extend(results)
        else:
            # No more tools to call - we're done
            done = True
    
    if iteration >= max_iterations:
        print("⚠️ Max iterations reached")
    
    return response.choices[0].message.content

## Step 8: Launch the Agent!

In [ ]:
# Create and launch Gradio interface
interface = gr.ChatInterface(
    chat,
    type="messages",
    title=f"Chat with {name}'s AI Agent",
    description=f"Ask me about {name}'s background, skills, and experience. I can also help you get in touch!",
    examples=[
        "What's your background?",
        "What technologies do you work with?",
        "Can we connect? My email is example@email.com",
        "What's your favorite food?"  # Will trigger unknown question tool
    ]
)

# Launch the interface
interface.launch()

## How It Works

### Architecture Flow:
```
User Message
    ↓
AI Generates Response + Tool Calls
    ↓
Execute Tools (send notifications)
    ↓
Send Tool Results Back to AI
    ↓
AI Generates Final Response
    ↓
Display to User
```

### Key Components:
1. **Tool Schemas**: Define available functions for AI
2. **Tool Handler**: Executes functions dynamically
3. **Pushover**: Sends real-time notifications
4. **Gradio UI**: Interactive web interface
5. **Agentic Loop**: Continues until no more tools needed

### Design Patterns Used:
- **Tool Use Pattern**: AI decides when to use tools
- **Notification Pattern**: Real-time alerts for important events
- **Dynamic Dispatch**: No hardcoded if/else for tools
- **Conversation Management**: Full context maintained

## Deployment to HuggingFace Spaces

Make your agent publicly accessible!

### Prerequisites:
1. Update files in `me/` folder with YOUR data
2. Change `name = "Your Name"` in the code above
3. Have a HuggingFace account

### Deployment Steps:

**1. Set up HuggingFace:**
- Visit https://huggingface.co and create an account
- Go to Settings → Access Tokens
- Create new token with WRITE permissions
- Save your token

**2. Install HuggingFace CLI:**
```bash
pip install huggingface_hub[cli]
```

**3. Login:**
```bash
huggingface-cli login
# Or: huggingface-cli login --token YOUR_TOKEN
```

**4. Deploy with Gradio:**
```bash
gradio deploy
```

**5. Follow prompts:**
- Space name: e.g., "career-assistant"
- Hardware: cpu-basic (free tier)
- Secrets: Add your API keys when prompted
  - OPENAI_API_KEY
  - PUSHOVER_USER
  - PUSHOVER_TOKEN

**6. Access your Space:**
- https://huggingface.co/spaces/YOUR_USERNAME/career-assistant

### Managing Your Space:

**Update Secrets:**
1. Go to your Space on HuggingFace
2. Click Settings (gear icon)
3. Scroll to "Variables and Secrets"
4. Add/edit/delete secrets

**Redeploy:**
```bash
gradio deploy
```

**Delete Space:**
1. Go to Space settings
2. Scroll to bottom
3. Click "Delete this Space"

## Extension Ideas

### Easy Additions:
1. **More Tools**:
   - `schedule_meeting` - Integrate with calendar API
   - `send_email` - Direct email integration
   - `log_to_database` - Store conversations in DB
   - `search_portfolio` - Search your projects

2. **Enhanced Notifications**:
   - Different priority levels
   - Rich notifications with links
   - Daily summary emails
   - Slack/Discord integration

3. **Analytics**:
   - Track popular questions
   - Monitor response quality
   - A/B test different prompts
   - User engagement metrics

### Advanced Features:
4. **Multi-Agent System**:
   - Specialist agents for different topics
   - Agent routing based on question type
   - Collaborative problem solving

5. **RAG Integration**:
   - Vector database for your content
   - Semantic search over your work
   - Dynamic context retrieval

6. **Quality Control**:
   - Add evaluator from previous lab
   - Self-correction loop
   - Confidence scoring

7. **Memory System**:
   - Remember past conversations
   - Build user profiles
   - Personalized responses

### Example: Adding a New Tool

```python
# 1. Define the function
def schedule_meeting(date, time, topic):
    """Schedule a meeting"""
    push(f"Meeting requested: {topic} on {date} at {time}")
    return {"scheduled": "ok"}

# 2. Define the schema
schedule_meeting_json = {
    "name": "schedule_meeting",
    "description": "Schedule a meeting",
    "parameters": {
        "type": "object",
        "properties": {
            "date": {"type": "string", "description": "Date (YYYY-MM-DD)"},
            "time": {"type": "string", "description": "Time (HH:MM)"},
            "topic": {"type": "string", "description": "Meeting topic"}
        },
        "required": ["date", "time"]
    }
}

# 3. Add to tools list
tools.append({"type": "function", "function": schedule_meeting_json})

# That's it! No need to modify handle_tool_calls!
```

## Tips for Success

### LinkedIn PDF:
- Export complete profile
- Ensure text is selectable
- Include all sections
- Keep it updated

### Summary.txt:
- Highlight unique achievements
- Include specific technologies
- Mention notable projects
- Keep under 500 words

### System Prompt:
- Define clear boundaries
- Set appropriate tone
- Include examples
- Test edge cases

### Tool Design:
- Keep functions simple
- Return structured data
- Handle errors gracefully
- Log important events

### Notifications:
- Be selective (avoid spam)
- Include context
- Make actionable
- Test thoroughly

## Troubleshooting

### "Pushover not working"
- Verify user key starts with 'u'
- Verify token starts with 'a'
- Check app is created on Pushover
- Install Pushover app on phone
- Test with curl: `curl -s --form-string "token=YOUR_TOKEN" --form-string "user=YOUR_USER" --form-string "message=test" https://api.pushover.net/1/messages.json`

### "Tools not being called"
- Check tool schemas are correct
- Verify system prompt mentions tools
- Look for finish_reason in logs
- Test with explicit requests

### "Gradio deploy fails"
- Ensure HuggingFace token has WRITE access
- Check you're logged in: `huggingface-cli whoami`
- Delete any existing README.md in folder
- Verify all files are present

### "Agent loops infinitely"
- Check max_iterations setting
- Verify tool returns proper format
- Look for errors in tool execution
- Add more logging

## Notes

- **Real-World Ready**: This is a production-quality agent
- **Extensible**: Easy to add new tools and features
- **Privacy**: Personal data protected by .gitignore
- **Monitoring**: Pushover provides real-time insights
- **Scalable**: Can handle multiple conversations
- **Professional**: Represents you 24/7